# HMM tagging

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
#from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import json
from ipywidgets import FloatProgress
from IPython.display import display
import numpy as np

In [2]:
train_set = json.load(open('corpus/fr/fr.ftb.train.json'))
# dev_set = json.load(open('corpus/fr/fr.pud.dev.json'))
test_set = json.load(open('corpus/fr/fr.ftb.test.json'))

In [3]:
def seperate_data_and_label(data_set):
    train_dataset = []
    train_labels = []
    for words, labels in data_set:
        train_dataset.append(words)
        train_labels.append(labels)
    return train_dataset, train_labels
train_dataset, train_labels = seperate_data_and_label(train_set)
test_dataset, test_labels = seperate_data_and_label(test_set)

In [4]:
def get_all_lables(data_set):
    res = set()
    for words, labels in data_set:
        res = res | set(labels)
    return list(res)
all_labels = get_all_lables(train_set)

In [10]:
all_labels

['ADV',
 'PART',
 'SCONJ',
 'PROPN',
 'AUX',
 'PUNCT',
 'PRON',
 'NUM',
 'NOUN',
 'X',
 'DET',
 'CCONJ',
 'ADP',
 'ADP+PRON',
 'ADP+DET',
 'INTJ',
 'VERB',
 'ADJ']

In [5]:
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    return Counter(sequences) 

sequences = []
for words in train_labels:
    sequences += words
tag_unigram = unigram_counts(sequences)

In [6]:
tag_unigram['NOUN']

93109

In [7]:
def bigram_counts(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """
    dict_freq = dict()
    for sequence in sequences:
        sequence_A, sequence_B = sequence[0:-1], sequence[1:]
        for tag in zip(sequence_A, sequence_B):
            if tag not in dict_freq.keys():
                dict_freq[tag] = 1
            else:
                dict_freq[tag] = dict_freq[tag] + 1
    return dict_freq
tag_bigram = bigram_counts(train_labels)

In [13]:
len(tag_bigram)

324

In [14]:
len(all_labels)

18

In [12]:
for tag1 in all_labels:
    for tag2 in all_labels:
        if not (tag1, tag2) in tag_bigram.keys():
            tag_bigram[(tag1, tag2)] = 0

In [15]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    dict_freq = dict()
    
    for tag in sequences:
       
        if tag[0] not in dict_freq.keys():
            dict_freq[tag[0]] = 1
        else:
            dict_freq[tag[0]] = dict_freq[tag[0]] + 1
            

    return dict_freq
tag_starts = starting_counts(train_labels)

In [16]:
tag_starts

{'ADV': 1136,
 'ADP+DET': 249,
 'DET': 4846,
 'NOUN': 856,
 'CCONJ': 747,
 'ADP': 2533,
 'PRON': 1678,
 'PROPN': 537,
 'PUNCT': 945,
 'ADJ': 289,
 'VERB': 501,
 'X': 21,
 'SCONJ': 228,
 'NUM': 142,
 'AUX': 30,
 'INTJ': 20,
 'ADP+PRON': 1}

In [17]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    dict_freq = dict()
    for tag in sequences:

        if tag[-1] not in dict_freq.keys():
            dict_freq[tag[-1]] = 1
        else:
            dict_freq[tag[-1]] = dict_freq[tag[-1]] + 1
            

    return dict_freq
tag_ends = ending_counts(train_labels)

In [18]:
tag_ends

{'PUNCT': 14454,
 'NOUN': 162,
 'ADJ': 76,
 'PROPN': 34,
 'VERB': 10,
 'NUM': 10,
 'ADV': 9,
 'X': 2,
 'PRON': 2}